In [4]:
import sqlite3
import pandas as pd

# get the data from the database
def extract_data(sqlite_file):
    conn = sqlite3.connect(sqlite_file)
    nvtx_df = pd.read_sql_query("SELECT start, end, textId FROM NVTX_EVENTS", conn)
    stringID_df = pd.read_sql_query("SELECT * FROM StringIds", conn)
    conn.close()
    
    nvtx_df['duration(ms)'] = nvtx_df['end'] - nvtx_df['start']
    nvtx_df['duration(ms)'] = nvtx_df['duration(ms)'] / 1000000  # 단위 변환(ns -> ms)
    
    nvtx_df = pd.merge(nvtx_df, stringID_df, left_on='textId', right_on='id', how='left')
    nvtx_df = nvtx_df.drop(['textId', 'id', 'start', 'end'], axis=1)
    nvtx_df = nvtx_df.rename(columns={'value': 'function_name'})
    
    # function_name을 기준으로 groupby 후 duration의 평균을 구함
    nvtx_df = nvtx_df.groupby(['function_name']).mean()
    
    # sort by function_name
    nvtx_df = nvtx_df.sort_values(by=['function_name'])
    
    # 원형 그래프로 시각화
    # nvtx_df.plot.pie(y='duration(ms)', figsize=(12, 5), autopct='%1.1f%%', xlabel='', ylabel='', legend=False)
    
    # total duration을 구함
    total_duration = nvtx_df['duration(ms)'].sum()
    # dataframe에 데이터 추가(function_name = "total_duration" duration = total_duration)
    nvtx_df.loc['total_duration'] = total_duration
    
    return nvtx_df


In [5]:
sqlite_file_path = "../../../baseline.sqlite"
baseline_df = extract_data(sqlite_file_path)
baseline_df

,duration(ms)
function_name,
3D_backbone,17.758560
bbox_head,3.087464
find_centers,67.653807
post_processing,16.189904
reader,3.727695
transformer_forward,20.196623
total_duration,128.614053


In [6]:
sqlite_file_path = "../../../poolformer.sqlite"
poolformer_df = extract_data(sqlite_file_path)
poolformer_df

,duration(ms)
function_name,
3D_backbone,15.046447
bbox_head,2.761143
find_centers,53.540357
poolformer_forward,0.460812
post_processing,6.320717
reader,1.360341
total_duration,79.489818
